# TensorFlow Recommenders: Matrix Factorization Model

TensorFlow Recommenders (TFRS)는 TensorFlow 기반의 추천 시스템 라이브러리로, 복잡한 추천 시스템을 쉽게 구축하고 실험할 수 있도록 도와줍니다. TFRS는 모듈화된 구성 요소, 다양한 손실 함수와 평가 지표, 데이터 전처리 도구 등을 제공하여 추천 시스템 개발을 단순화합니다. Matrix Factorization 모델은 TFRS를 사용하여 구현할 수 있는 대표적인 모델로, 사용자와 아이템 임베딩을 통해 추천을 수행합니다.

https://www.tensorflow.org/recommenders/examples/quickstart

TFRS와 함께 [MovieLens 100K 데이터 세트](https://grouplens.org/datasets/movielens/100k/)를 사용하여 간단한 행렬 분해(matrix factorization) 모델을 구축합니다. 이 모델을 사용하여 특정 사용자에게 영화를 추천할 수 있습니다.

In [ ]:
# TFRS와 호환되는 이전 version 설치

In [ ]:
# 타입 힌트를 위한 라이브러리 임포트
# 다양한 데이터셋을 쉽게 로드할 수 있는 라이브러리
# 추천 시스템을 구축하기 위한 TensorFlow 라이브러리
# 데이터를 보기 좋게 출력하기 위한 라이브러리

### Read the data

In [ ]:
# 영화 평점 데이터를 불러옵니다.
# 모든 사용 가능한 영화의 특성 데이터를 불러옵니다.

In [ ]:
# ratings 데이터셋에서 첫 번째 요소를 가져와 출력
# 줄바꿈을 위해 빈 줄 출력
# movies 데이터셋에서 첫 번째 요소를 가져와 출력

In [ ]:
# 'ratings' 데이터셋에서 필요한 특성만 선택하여 새로운 구조로 매핑합니다.
# 이 과정에서 'movie_title'과 'user_id' 필드만을 포함시킵니다.
# 'movies' 데이터셋에서 'movie_title' 필드만을 선택합니다.

In [ ]:
# ratings 데이터셋에서 첫 번째 요소를 가져와 출력
# 줄바꿈을 위해 빈 줄 출력
# movies 데이터셋에서 첫 번째 요소를 가져와 출력

embedding layer를 위해 `사용자 ID`와 `영화 제목`을 정수 인덱스로 변환하는 vocabulary를 구축합니다.  

- `StringLookup` 레이어는 텍스트 데이터를 처리할 때 유용하게 사용됩니다. 이 경우, 사용자 ID와 영화 제목을 각각의 정수 인덱스로 매핑하여 모델이 이해할 수 있는 형태로 변환하는 역할을 합니다.  
-`adapt` 메소드는 주어진 데이터셋을 기반으로 어휘 사전을 자동으로 구성합니다. 이 과정에서 데이터셋의 모든 고유한 문자열 값을 스캔하여 인덱스를 할당합니다.  
- `mask_token=None` 옵션은 입력 데이터 중 어떤 값도 마스킹(무시)하지 않고 모두 처리하겠다는 것을 의미합니다. 일부 경우에는 특정 값을 무시하고 싶을 때 mask_token을 다르게 설정할 수 있습니다.  

이러한 어휘 사전을 사용하면 모델이 문자열 형태의 데이터를 쉽게 처리할 수 있으며, 추후에 이 데이터를 모델의 입력으로 사용할 때 일관된 방식으로 인코딩할 수 있습니다.

In [ ]:
# 사용자 ID를 위한 어휘 사전 생성. mask_token=None은 어떤 토큰도 마스킹하지 않겠다는 의미입니다.
# ratings 데이터에서 사용자 ID를 추출하여 어휘 사전에 적용(adapt)하여 인덱싱합니다.
# 영화 제목을 위한 어휘 사전 생성. 여기서도 mask_token=None으로 설정합니다.
# movies 데이터셋을 사용하여 영화 제목에 대한 어휘 사전을 구성합니다.

In [ ]:
# 사용자 id list

In [ ]:
# 영화 title list

In [ ]:
# TensorFlow constant로  사용자 ID 3개를 생성합니다.
# 생성된 어휘 사전을 사용하여 데이터에 포함된 사용자 ID를 정수 인덱스로 변환합니다.
# 변환된 인덱스는 모델의 입력으로 사용될 수 있습니다.

In [ ]:
# TensorFlow constant로 영화 제목 데이터를 3개 생성합니다.
# 생성된 영화 제목 어휘 사전을 사용하여 주어진 영화 제목들을 정수 인덱스로 변환합니다.
# 이 변환된 데이터는 모델의 입력으로 사용될 수 있으며, 각 영화 제목이 어휘 사전에서 어떤 인덱스로 할당되었는지 확인할 수 있습니다.

### model 정의

`tfrs.Model`을 상속하고 `compute_loss` 메서드를 구현하여 TFRS 모델을 정의할 수 있습니다.

In [ ]:
class MovieLensModel(tfrs.Model):
  def __init__(self, user_model: tf.keras.Model,
    # 사용자와 영화 표현(representation) 설정
    # 검색(retrieval) 작업 설정
  # 손실 함수(loss function) 계산을 위한 메소드
  # Dict[Text, tf.Tensor] --> 문자열 키와 Tensor값으로 구성된 dictionary
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # 사용자 ID로부터 사용자 임베딩 생성
    # 영화 제목으로부터 영화 임베딩 생성
    # 생성된 사용자와 영화 임베딩을 이용하여 검색 작업을 수행하고, 손실을 반환

두 모델과 retrieval task를 정의합니다.

tfrs.tasks.Retrieval: 이는 TFRS에서 제공하는 검색(retrieval) 작업을 설정하는 클래스입니다. 검색 작업은 사용자에게 관련 아이템을 추천하는 모델의 능력을 평가하고 최적화하는 데 중점을 둡니다.

tfrs.metrics.FactorizedTopK: 이 메트릭은 모델이 얼마나 잘 사용자에게 상위 K개의 관련 아이템을 추천하는지 평가합니다. 즉, 모델이 생성한 아이템 임베딩들 중 사용자의 실제 선호와 가장 유사한 아이템을 정확하게 찾아낼 수 있는지를 측정합니다.

movies.batch(128).map(movie_model): 이 부분은 FactorizedTopK 메트릭에 전달되는 아이템 후보 데이터셋을 준비하는 과정입니다. movies 데이터셋을 배치 크기 128로 나누고, 각 배치에 대해 movie_model을 적용하여 영화 임베딩을 생성합니다. 이렇게 생성된 임베딩들은 메트릭 계산에 사용되는 아이템 후보군으로 활용됩니다.


In [ ]:
# 사용자 모델 정의 - 사용자 임베딩 생성
# 이 모델은 사용자 ID를 입력으로 받아 임베딩 벡터를 출력합니다.
# 영화 모델 정의 - 영화 임베딩 생성
# 이 모델은 영화의 제목을 입력으로 받아 임베딩 벡터를 출력합니다.
# 추천 시스템의 목적(목표) 함수 정의
# 생성된 영화 임베딩을 사용하여, 모델이 주어진 사용자에 대해 얼마나 관련성 높은 영화를 상위 K개로 추천할 수 있는지 평가합니다.
# FactorizedTopK 메트릭을 통해 계산된 성능 지표를 기반으로 모델을 학습하고 최적화합니다.
# 목표는 이 메트릭의 값을 최대화하는 것입니다.

영화 제목에 대한 임베딩을 직접 가져올 수 있습니다

In [ ]:
# 학습된 영화 모델을 사용하여 "Star Wars (1977)" 영화의 임베딩 벡터를 예측합니다.
# 이 함수는 모델에 문자열 데이터를 입력으로 제공하고, 모델은 이 입력에 대한 영화의 특징을 나타내는 임베딩 벡터를 출력합니다.
# 예측된 임베딩 벡터의 차원을 출력합니다.
# 예측된 임베딩 벡터를 출력합니다.
# 이 벡터는 영화 "Star Wars (1977)"의 특징을 수치적으로 표현한 것이며, 추천 시스템 등에서 사용될 수 있습니다.


### Fit and evaluate it.

`tf.recommenders.layers.factorized_top_k`는 TensorFlow Recommenders (TFRS) 라이브러리 내에 있는 레이어 모음으로, 추천 시스템에서 사용자에게 아이템을 추천할 때 상위 K개의 가장 관련성 높은 아이템을 찾아내는 기능을 제공합니다. 이 레이어들은 추천 모델의 평가 단계에서 주로 사용되며, 모델이 얼마나 잘 사용자의 선호도를 예측하는지를 측정하는 데 도움을 줍니다.

### `factorized_top_k` 의 item 검색 방법:

1. **BruteForce**:  모든 아이템 후보에 대해 순차적으로 사용자 쿼리와의 유사성(또는 거리)을 계산하고, 가장 유사한 상위 K개의 아이템을 반환. 이 과정은 계산적으로 비효율적일 수 있지만, 작은 데이터셋에서는 실용적일 수 있습니다.  

    - index_from_dataset 메서드는 TFRS 라이브러리의 BruteForce 레이어에서 제공하는 메서드로, 데이터셋에서 항목을 로드하여 인덱싱하는 데 사용됩니다. 이 메서드는 특히 추천 시스템에서 사용되며, 학습된 모델을 사용하여 생성된 임베딩을 인덱스 구조에 추가하여, 추후 검색(쿼리) 작업 시 빠르고 효율적으로 유사 항목을 찾을 수 있게 해줍니다.

2. **ScaNN**: 대규모 데이터셋에서 더 효율적인 근사 최근접 이웃 검색을 수행하는 레이어입니다.



In [ ]:
# 사용자 모델과 영화 모델을 결합한 검색 모델을 생성합니다.
# TFRS (TensorFlow Recommenders) 라이브러리를 활용하여 추천 모델을 구현합니다.
# 모델을 컴파일합니다. 손실 함수를 지정하지 않으면, TFRS 모델은 task 객체에 정의된 내장 손실 함수를 사용합니다.
# Adagrad 최적화 알고리즘을 사용하며 학습률은 0.5로 설정합니다.
# 모델을 3 에폭 동안 훈련합니다. 배치 크기는 4096으로 설정합니다.
# 훈련된 모델을 사용하여 무작위 탐색(brute-force search)을 설정합니다.
# 이를 통해 사용자에게 영화 추천을 수행할 수 있습니다.
# 영화 데이터셋을 사용하여 무작위 탐색 인덱스를 생성합니다.
# 여기서는 영화 제목을 입력으로 받아 영화 모델이 반환한 임베딩을 인덱스에 추가합니다.

In [ ]:
# 특정 사용자에 대한 추천 받기

In [ ]:
# 특정 사용자에 대한 추천 받기
# 상위 3개 추천 영화 타이틀을 문자열 리스트로 변환하여 출력